# Вариант 3. Квантование весов для NPU

Реализация квантования весов нейронной сети из float32 в int8 для использования в NPU:
- функция квантования весов;
- класс `QuantizedLayer` с хранением квантованных весов и forward с деквантованием;
- сравнение точности до и после квантования на простых данных.

In [ ]:
# Импорты
from __future__ import annotations

import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

## 1. Функция квантования весов float32 → int8

Симметричное квантование: масштаб по максимуму абсолютных значений, веса приводятся к диапазону int8 [-128, 127].

In [ ]:
def quantize_weights(weights_float: np.ndarray) -> tuple[np.ndarray, float]:
    """
    Квантует веса из float32 в int8 для использования в NPU (симметричное квантование).

    Масштаб: scale = max(|weights|) / 127. Значения округляются и ограничиваются [-128, 127].

    Parameters
    ----------
    weights_float : np.ndarray
        Массив весов в float32, форма (out_features, in_features) или любая 2D.

    Returns
    -------
    tuple[np.ndarray, float]
        (weights_int8, scale): квантованные веса в int8 и масштаб для деквантования.
    """
    w = np.asarray(weights_float, dtype=np.float64)
    w_max = np.max(np.abs(w))
    if w_max == 0:
        scale = 1.0
        weights_int8 = np.zeros(w.shape, dtype=np.int8)
        return weights_int8, scale
    scale = w_max / 127.0
    weights_int8 = np.round(w / scale).astype(np.float64).clip(-128, 127).astype(np.int8)
    return weights_int8, float(scale)


def dequantize_weights(weights_int8: np.ndarray, scale: float) -> np.ndarray:
    """
    Восстанавливает веса из int8 в float для вычислений (деквантование).

    Parameters
    ----------
    weights_int8 : np.ndarray
        Квантованные веса, dtype int8.
    scale : float
        Масштаб, использованный при квантовании.

    Returns
    -------
    np.ndarray
        Веса в float64 (приближение исходных float32).
    """
    return weights_int8.astype(np.float64) * scale

## 2. Класс QuantizedLayer

Хранит квантованные веса (int8) и масштаб; при forward выполняет деквантование и линейное преобразование.

In [ ]:
class QuantizedLayer:
    """
    Линейный слой с квантованными весами (int8) для NPU.
    Хранит веса в int8 и при forward деквантует их на лету и выполняет Y = X @ W^T.
    """

    def __init__(self, weights_float: np.ndarray):
        """
        Инициализация: квантует переданные веса float32 и сохраняет int8 + scale.

        Parameters
        ----------
        weights_float : np.ndarray
            Матрица весов (out_features, in_features) в float32.
        """
        self._weights_int8, self._scale = quantize_weights(weights_float)
        self._out_features, self._in_features = self._weights_int8.shape

    @property
    def weights_int8(self) -> np.ndarray:
        """Квантованные веса (int8)."""
        return self._weights_int8

    @property
    def scale(self) -> float:
        """Масштаб для деквантования."""
        return self._scale

    def forward(self, x: np.ndarray) -> np.ndarray:
        """
        Прямое распространение: деквантование весов и умножение x @ W^T.

        Parameters
        ----------
        x : np.ndarray
            Входной батч, форма (batch_size, in_features).

        Returns
        -------
        np.ndarray
            Выход формы (batch_size, out_features).
        """
        w = dequantize_weights(self._weights_int8, self._scale)
        return x @ w.T

## 3. Сравнение точности до и после квантования

Используем простую регрессию: данные из `make_regression`, линейная модель на float32 (через явные веса), затем те же веса квантуем в `QuantizedLayer` и сравниваем MSE и MAE на тесте.

In [ ]:
# Генерация простых данных и обучение "модели" (линейная регрессия через нормальное уравнение)
np.random.seed(42)
n_samples, n_features = 500, 20
X, y = make_regression(n_samples=n_samples, n_features=n_features, noise=5.0, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
# Добавляем столбец единиц для bias
X_train_b = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
X_test_b = np.hstack([X_test, np.ones((X_test.shape[0], 1))])

# Веса методом нормальных уравнений: W = (X'X)^(-1) X' y  -> (n_features+1,) для одной цели
# Для единообразия с QuantizedLayer храним веса как (1, in_features)
w_opt = np.linalg.lstsq(X_train_b, y_train.reshape(-1, 1), rcond=None)[0]
weights_float = w_opt.T  # (1, n_features+1)

In [ ]:
# Предсказания до квантования (float32)
y_pred_float = (X_test_b @ weights_float.T).flatten()
mse_float = mean_squared_error(y_test, y_pred_float)
mae_float = mean_absolute_error(y_test, y_pred_float)
print("До квантования (float32):")
print(f"  MSE = {mse_float:.4f}, MAE = {mae_float:.4f}")

In [ ]:
# Квантованный слой и предсказания после квантования
qlayer = QuantizedLayer(weights_float)
y_pred_quant = qlayer.forward(X_test_b).flatten()
mse_quant = mean_squared_error(y_test, y_pred_quant)
mae_quant = mean_absolute_error(y_test, y_pred_quant)
print("После квантования (int8 + деквантование в forward):")
print(f"  MSE = {mse_quant:.4f}, MAE = {mae_quant:.4f}")

In [ ]:
# Сводка сравнения
print("\n--- Сравнение точности ---")
print(f"  MSE:  float32 = {mse_float:.4f}, int8 = {mse_quant:.4f}, относительное изменение = {(mse_quant - mse_float) / (mse_float + 1e-10) * 100:+.2f}%")
print(f"  MAE:  float32 = {mae_float:.4f}, int8 = {mae_quant:.4f}, относительное изменение = {(mae_quant - mae_float) / (mae_float + 1e-10) * 100:+.2f}%")
print("\nКвантование в int8 даёт небольшое ухудшение метрик из-за округления; для NPU это компромисс между точностью и скоростью/энергоэффективностью.")

In [ ]:
# Дополнительно: проверка деквантованных весов (ошибка квантования)
w_restored = dequantize_weights(qlayer.weights_int8, qlayer.scale)
weight_error = np.abs(weights_float.astype(np.float64) - w_restored)
print("Ошибка восстановления весов после квантования:")
print(f"  max |w_float - w_dequant| = {np.max(weight_error):.6f}")
print(f"  mean |w_float - w_dequant| = {np.mean(weight_error):.6f}")